In [1]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [23]:
import boto3 
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

In [8]:
import json
import os
import sys

import boto3


import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

module_path = ".."
sys.path.append(os.path.abspath(module_path))

#Condensing data 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Sample transcript data
transcript = """
Agent: Hello, thank you for calling our support line. How can I assist you today?
Patient: Hi, yes, I'm calling about my recent prescription.
Agent: Sure, I'd be happy to help. Can you please provide your name and date of birth?
Patient: My name is John Doe, and my date of birth is January 1st, 1980.
Agent: Thank you, John. What seems to be the issue with your prescription?
Patient: Well, I think there might be a mistake in the dosage instructions.
Agent: Alright, let me pull up your prescription information. Could you please confirm the medication name and dosage?
Patient: It's for medication X, and I'm supposed to take 2 tablets twice a day.
Agent: Okay, let me check that for you.
"""

# Define a list of common greetings
greetings = ['hello', 'hi', 'hey', 'howdy','Okay','happy',',']

transcript_list = transcript.split("\n")

# Tokenize the transcript
tokens = []
for i in range(len(transcript_list)):
    token = word_tokenize(transcript_list[i].lower())
    tokens.append(token)
#tokens = word_tokenize(transcript.lower())

filtered_tokens = []
for i in range(len(tokens)):
    str1 = ""
    for j in range(len(tokens[i])):
        #if tokens[i][j] not in stopwords.words('english') and tokens[i][j] not in greetings:
        if tokens[i][j] not in greetings:
            if j == 0:
                str1 += tokens[i][j]
            else:
                str1 += " "
                str1 += tokens[i][j]
    filtered_tokens.append(str1)
# Filter out common greetings and stopwords
#filtered_tokens = [word for word in tokens if word not in stopwords.words('english') and word not in greetings]

for i in range(len(filtered_tokens)):
    print(filtered_tokens[i])
# Reconstruct the filtered transcript
#filtered_transcript = ' '.join(filtered_tokens)

#print(filtered_transcript)

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data] Downloading package omw-1.4 to /home/ec2-user/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...



agent : thank you for calling our support line . how can i assist you today ?
patient : yes i 'm calling about my recent prescription .
agent : sure i 'd be to help . can you please provide your name and date of birth ?
patient : my name is john doe and my date of birth is january 1st 1980 .
agent : thank you john . what seems to be the issue with your prescription ?
patient : well i think there might be a mistake in the dosage instructions .
agent : alright let me pull up your prescription information . could you please confirm the medication name and dosage ?
patient : it 's for medication x and i 'm supposed to take 2 tablets twice a day .
agent : okay let me check that for you .



[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
filtered_tokens

['',
 'agent : thank you for calling our support line . how can i assist you today ?',
 "patient : yes i 'm calling about my recent prescription .",
 "agent : sure i 'd be to help . can you please provide your name and date of birth ?",
 'patient : my name is john doe and my date of birth is january 1st 1980 .',
 'agent : thank you john . what seems to be the issue with your prescription ?',
 'patient : well i think there might be a mistake in the dosage instructions .',
 'agent : alright let me pull up your prescription information . could you please confirm the medication name and dosage ?',
 "patient : it 's for medication x and i 'm supposed to take 2 tablets twice a day .",
 'agent : okay let me check that for you .',
 '']

# Text Summarization

In [12]:
pip install meta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for meta: filename=meta-1.0.2-py3-none-any.whl size=59235 sha256=ccd88988f84673ceca70a1d5511ad0746fadbd27a06fd0e273bdb96346b49f78
  Stored in directory: /home/ec2-user/.cache/pip/wheels/92/39/0b/1c4852017b6fd8a642c0481ade113ebd0b0d14e3d06d631593
Successfully built meta
Note: you may need to restart the kernel to use updated packages.


In [14]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [15]:
!huggingface-cli login‍

usage: huggingface-cli <command> [<args>]
huggingface-cli: error: argument {env,login,whoami,logout,repo,upload,download,lfs-enable-largefiles,lfs-multipart-upload,scan-cache,delete-cache}: invalid choice: 'login\u200d' (choose from 'env', 'login', 'whoami', 'logout', 'repo', 'upload', 'download', 'lfs-enable-largefiles', 'lfs-multipart-upload', 'scan-cache', 'delete-cache')


In [71]:
def text_summariser_llama2(bedrock_runtime , prompt , temp , top_p):
    try:
        body = {
            "prompt": prompt,
            "temperature": temp,
            "top_p": top_p,
            "max_gen_len": 200
            }

        response = bedrock_runtime.invoke_model(
            modelId="meta.llama2-13b-chat-v1", body=json.dumps(body)
        )

        response_body = json.loads(response["body"].read())
        completion = response_body["generation"]

        return completion

    except ClientError:
        logger.error("Couldn't invoke Llama 2")
        raise

In [27]:
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.1 MB/s eta 0:00:00


In [28]:
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from bert_score import BERTScorer

def bert_scorer(reference,candidate):
    scorer = BERTScorer(model_type='bert-base-uncased')
    P, R, F1 = scorer.score([candidate], [reference])
    return f"BERTScore Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}"

In [51]:
example_1 = '''
Agent: Hello, thank you for calling our support line. How can I assist you today?
Patient: Hi, yes, I'm calling about my recent prescription.
Agent: Sure, I'd be happy to help. Can you please provide your name and date of birth?
Patient: My name is John Doe, and my date of birth is January 1st, 1980.
Agent: Thank you, John. What seems to be the issue with your prescription?
Patient: Well, I think there might be a mistake in the dosage instructions.
Agent: Alright, let me pull up your prescription information. Could you please confirm the medication name and dosage?
Patient: It's for medication X, and I'm supposed to take 2 tablets twice a day.
Agent: Okay, let me check that for you.
'''

example_summary_original = "The patient, John Doe, is calling the support line about a prescription issue. He believes there might be a mistake in the dosage instructions. The agent confirms the medication name and dosage for medication X, which is to take 2 tablets twice a day. The agent then checks the information for the patient."

In [47]:
candidate = '''
Agent : Good afternoon, thank you for calling ABC Company customer service.My name is Sarah, how can I assist you today? 
Patient : Hi Sarah, my name is John Smith.I'm calling because I haven't received my order yet, and it's been over a week since I placed it. 
Agent: I apologize for the inconvenience, John. Can you please provide me with your order number so I can look into this for you? 
Patient : Sure, it's #123456789. 
Agent : Thank you, John. Let me check on the status of your order. It looks like your package was delayed due to a backlog at our shipping facility. I'm really sorry about that. 
Patient : That's frustrating, but I understand. Is there an estimated delivery date for my order now? 
Agent : Yes, according to our system, your package is scheduled to be delivered by Friday of this week. Would you like me to expedite the shipping at no extra cost for the inconvenience? 
Patient: That would be great, thank you. I appreciate your help with this. 
AGent : Of course, John. I've upgraded your shipping to expedited, and you should receive a confirmation email shortly. Is there anything else I can assist you with? **Customer:** No, that's all for now. Thank you for your help, Sarah. 
Agent : You're welcome, John. If you have any further questions or concerns, don't hesitate to reach out. Thank you for choosing ABC Company, and have a great day!
''' 

summary_original = "John called ABC Company's customer service regarding a delayed order. CSR Sarah apologized for the delay and offered to expedite the shipping at no extra cost. John agreed, and Sarah upgraded the shipping method. John expressed gratitude, and the call ended with Sarah ensuring John's satisfaction and offering further assistance if needed."


In [69]:
import boto3
import json
from botocore.exceptions import ClientError

def invoke_llama2(bedrock_runtime_client, prompt):
        """
        Invokes the Meta Llama 2 large-language model to run an inference
        using the input provided in the request body.

        :param prompt: The prompt that you want Jurassic-2 to complete.
        :return: Inference response from the model.
        """

        try:
            # The different model providers have individual request and response formats.
            # For the format, ranges, and default values for Meta Llama 2 Chat, refer to:
            # https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-meta.html

            body = {
                "prompt": prompt,
                "temperature": 0.5,
                "top_p": 0.9,
                "max_gen_len": 512,
            }

            # model_id = 'meta.llama2-13b-chat-v1'
            model_id = 'meta.llama2-70b-chat-v1'
            response = bedrock_runtime_client.invoke_model(
                modelId=model_id, body=json.dumps(body)
            )

            response_body = json.loads(response["body"].read())
            completion = response_body["generation"]

            return completion

        except ClientError:
            logger.error("Couldn't invoke Llama 2")
            raise

# management plane, use bedrock
# brt = boto3.client(service_name='bedrock')
# brt.list_foundation_models()
brt = boto3.client(service_name='bedrock-runtime')
result = invoke_llama2(brt, 'what is llama 2?')
print(result)


Llama 2 is a new, more powerful, and more efficient language model developed by Meta AI. It is the successor to the original Llama language model, which was introduced in 2017. Llama 2 is designed to handle more complex and open-ended natural language tasks, such as conversational dialogue and text generation, and it is trained on a larger and more diverse dataset than the original Llama.

Llama 2 uses a transformer architecture, which is a type of neural network that is particularly well-suited for natural language processing tasks. The model is trained on a large dataset of text from the internet, and it is designed to learn the patterns and structures of natural language. This allows it to generate coherent and contextually appropriate text, and to engage in conversation in a way that is natural and human-like.

One of the key advantages of Llama 2 is its ability to handle multi-turn dialogue, which means that it can engage in conversations that involve multiple exchanges of inform

In [79]:
prompt1 = f"""
Instruction: "Summarise this call transcript between a patient and an agent and provide it in a precise paragraph :" \n : 
{candidate}.
             
Response : 
           """

brt = boto3.client(service_name='bedrock-runtime')
summary_1 = text_summariser_llama2(brt,prompt1,0,0.5)
summary_1

"\nThe patient, John Smith, called ABC Company's customer service to inquire about his order, which had been delayed for over a week. The agent, Sarah, apologized for the inconvenience and asked for the order number to investigate. After checking the status, Sarah explained that the package was delayed due to a backlog at the shipping facility, but assured John that it would be delivered by the end of the week. She also offered to expedite the shipping at no extra cost to make up for the delay. John accepted the offer and thanked Sarah for her help."

In [62]:
template = f"""
Instruction: "Summarise this call transcript between a patient and an agent and provide it in a precise paragraph :" \n : 
{candidate}.
             
Response : 
           """
print(template)

Summarise this call transcript between a patient and an agent and provide it in a precise paragraph : 
             
Agent : Good afternoon, thank you for calling ABC Company customer service.My name is Sarah, how can I assist you today? 
Patient : Hi Sarah, my name is John Smith.I'm calling because I haven't received my order yet, and it's been over a week since I placed it. 
Agent: I apologize for the inconvenience, John. Can you please provide me with your order number so I can look into this for you? 
Patient : Sure, it's #123456789. 
Agent : Thank you, John. Let me check on the status of your order. It looks like your package was delayed due to a backlog at our shipping facility. I'm really sorry about that. 
Patient : That's frustrating, but I understand. Is there an estimated delivery date for my order now? 
Agent : Yes, according to our system, your package is scheduled to be delivered by Friday of this week. Would you like me to expedite the shipping at no extra cost for the i

In [64]:
text_summariser_llama2(template,0.9,0.9)

''

In [82]:
temp = [0,0.5,0.9]
top_p = [0.1,0.5,0.9]

#prompt = f"""Summarise this call transcript between a patient and an agent and provide it in a precise paragraph : {candidate}"""
prompt_llama = f"""
Instruction: "Summarise this call transcript between a patient and an agent and provide it in a precise paragraph :" \n : 
{example_1}.
             
Response : 
           """
hyper_parameters_tuning_scores = {}
hyper_parameters_tuning_text_summaries = {}
brt = boto3.client(service_name='bedrock-runtime')

for t in temp:
    str1 = ""
    for p in top_p:
        str1 = f"[Temperate : {t} , top_p : {p}]"
        summary = text_summariser_llama2(brt,prompt_llama,t,p)
        scores = bert_scorer(example_summary_original,summary)
        hyper_parameters_tuning_text_summaries[str1] = summary 
        hyper_parameters_tuning_scores[str1] = scores

In [86]:
keys1 = list(hyper_parameters_tuning_text_summaries.keys())
for i in range(len(hyper_parameters_tuning_text_summaries)):
    print("\n")
    print(f"{keys1[i]} : ")
    print(f"{hyper_parameters_tuning_text_summaries[keys1[i]]}")



[Temperate : 0 , top_p : 0.1] : 

Agent: John, I apologize for any inconvenience. It looks like there was indeed a mistake in the dosage instructions. The correct dosage for medication X is 1 tablet twice a day. I've updated the prescription information with the correct dosage. Would you like me to send a new prescription to your pharmacy?
Patient: Yes, please. And can you also confirm that the medication is covered by my insurance?
Agent: Yes, medication X is covered under your insurance plan. I've confirmed the coverage with your insurance provider. Would you like me to send the new prescription to your pharmacy now?
Patient: Yes, please go ahead and do that. Thank you so much for your help.
Agent: You're welcome, John. Is there anything else I can assist you with today?
Patient: No, that's all. Thank you again


[Temperate : 0 , top_p : 0.5] : 

Agent: John, I apologize for any inconvenience. It looks like there was indeed a mistake in the dosage instructions. The correct dosage f

In [85]:
keys2 = list(hyper_parameters_tuning_scores.keys())
for i in range(len(hyper_parameters_tuning_text_summaries)):
    print(f"{keys2[i]} : ")
    print(f"{hyper_parameters_tuning_scores[keys2[i]]}")

[Temperate : 0 , top_p : 0.1] : 
BERTScore Precision: 0.4778, Recall: 0.6425, F1: 0.5480
[Temperate : 0 , top_p : 0.5] : 
BERTScore Precision: 0.4778, Recall: 0.6425, F1: 0.5480
[Temperate : 0 , top_p : 0.9] : 
BERTScore Precision: 0.4987, Recall: 0.6560, F1: 0.5666
[Temperate : 0.5 , top_p : 0.1] : 
BERTScore Precision: 0.4910, Recall: 0.6516, F1: 0.5600
[Temperate : 0.5 , top_p : 0.5] : 
BERTScore Precision: 0.4778, Recall: 0.6425, F1: 0.5480
[Temperate : 0.5 , top_p : 0.9] : 
BERTScore Precision: 0.7904, Recall: 0.7568, F1: 0.7733
[Temperate : 0.9 , top_p : 0.1] : 
BERTScore Precision: 0.4987, Recall: 0.6560, F1: 0.5666
[Temperate : 0.9 , top_p : 0.5] : 
BERTScore Precision: 0.4977, Recall: 0.6400, F1: 0.5600
[Temperate : 0.9 , top_p : 0.9] : 
BERTScore Precision: 0.7877, Recall: 0.7964, F1: 0.7920


The Best Value of Temperate is 0.5 and top_p is 0.9

# QnA

In [87]:
prompt_llama = f"""
Instruction: "Summarise this call transcript between a patient and an agent and provide it in a precise paragraph :" \n : 
{example_1}.
             
Response : 
           """

final_summary = text_summariser_llama2(brt,prompt_llama,0.5,0.9)

In [92]:
print(final_summary)


The patient, John Doe, called the support line to inquire about his recent prescription. He mentioned that there might be a mistake in the dosage instructions. The agent confirmed the medication name and dosage and checked the prescription information. 


In [2]:
import boto3 
bedrock = boto3.client(
    service_name='bedrock', 
    region_name='us-east-1'
)

bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '4b562443-bd40-4375-9933-234743d3970c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 13 Feb 2024 12:27:12 GMT',
   'content-type': 'application/json',
   'content-length': '17098',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4b562443-bd40-4375-9933-234743d3970c'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
   'modelId': 'amazon.titan-image-generator-v1:0',
   'modelName': 'Titan Image Generator G1',
   'providerName': 'Amazon',

In [4]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 8.2 MB/s eta 0:00:00
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached typing_extensions-4.9.0-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 10.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 50.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00
Using cached packaging-23.2-py3-none-any.whl (53 kB)
Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.

In [89]:
print(final_summary)


The patient, John Doe, called the support line to inquire about his recent prescription. He mentioned that there might be a mistake in the dosage instructions. The agent confirmed the medication name and dosage and checked the prescription information. 


In [91]:
import boto3
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms import Bedrock

bedrock_runtime_claude = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

claude_prompt = f"""
Human : You are an agent assistant who helps the human customer care agent answer the patient's/customer's query by using
the information from the following Current conversation summary and providing a satisfactory answer to the patient/customer.

Current conversation summary:
<conversation_history>
{final_summary}
</conversation_history>

Here is the human's next reply:
<human_reply>
{input}
</human_reply>

Assistant:
"""

model_id = "anthropic.claude-v2"
model_kwargs =  { 
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
    "prompt": claude_prompt,
}

llm = Bedrock(
    client=bedrock_runtime,
    model_id=model_id,
    model_kwargs=model_kwargs
)

conversation= ConversationChain(
    llm=llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

conversation.predict(input="Hey,before I can end the conversation,are there any other change in dosage instruction?")

" I'm afraid I don't have any additional information about dosage instructions beyond what we've already discussed. Since this was a friendly conversation without a specific medical context, I don't have any details about medications or dosage instructions to share. I'm limited in what I can say without a proper medical context. Let me know if you have any other questions!"

# Testing the Pipeline

In [95]:
import boto3
import pandas as pd

# Initialize the S3 client
s3 = boto3.client('s3')

# Specifying the bucket name and CSV file key
bucket_name = 'agentassistuc2'
file_key = 'Call_Conv_2_12.txt'

# Specifying the file path where you want to save the CSV data locally (here in the sagemaker instance's jupyter lab)
local_file_path = 'Call_Conv_Test_1.txt'

# Download the file from S3 to a local file
s3.download_file(bucket_name, file_key, local_file_path)

In [118]:
# Now you can work with the local text file as needed
with open(local_file_path, 'r') as file:
    text_data = file.read()

In [119]:
print(text_data)

[
  {
    Transcript: {
      ParticipantId: 'AGENT',
      ParticipantRole: 'AGENT',
      Content: 'Improving. Uh, Okay? Mm hmm. Obviously.',
      BeginOffsetMillis: 1257,
      EndOffsetMillis: 9697,
      Sentiment: 'NEUTRAL',
      Id: 'fb29489e-d06b-48b1-8e0b-519f17c4a68e'
    }
  },
  {
    Transcript: {
      ParticipantId: 'CUSTOMER',
      ParticipantRole: 'CUSTOMER',
      Content: 'Fine. Thank you. I think',
      BeginOffsetMillis: 3047,
      EndOffsetMillis: 8055,
      Sentiment: 'POSITIVE',
      Id: '7f6820ee-0487-4cbc-9b35-33e7c31e1d5d'
    }
  },
  {
    Transcript: {
      ParticipantId: 'CUSTOMER',
      ParticipantRole: 'CUSTOMER',
      Content: 'A little bit. More even now.',
      BeginOffsetMillis: 10457,
      EndOffsetMillis: 16775,
      Sentiment: 'POSITIVE',
      Id: '43d73132-0645-4133-8488-26aa9eac95bc'
    }
  },
  {
    Transcript: {
      ParticipantId: 'AGENT',
      ParticipantRole: 'AGENT',
      Content: "Yeah, you're audible and disconnecting

In [142]:
import json

def preprocess_text_data(text_data):
    # Replace single quotes with double quotes
    text_data = text_data.replace("'", '"')
    
    # Add double quotes around keys
    text_data = text_data.replace("{", '{"').replace(":", '":')
    
    return text_data

def extract_content_from_transcripts(text_data):
    try:
        text_data = preprocess_text_data(text_data)
        
        # Convert the text data to a list of dictionaries
        transcript_list = json.loads(text_data)

        # Initialize an empty list to store content from each transcript
        content_list = []

        # Iterate through each transcript dictionary and extract the content
        for transcript in transcript_list:
            str1 = ""
            content = transcript.get('Transcript', {}).get('Content')
            role = transcript.get('Transcript',{}).get('ParticipantRole')
            str1 = str1 + role + " : " + content
            if content:
                content_list.append(str1)
        
        call_transcript = ""
        for i in range(len(content_list)):
            if i != len(content_list)-1:
                call_transcript += content_list[i]
                call_transcript += "\n"
            else:
                call_transcript += content_list[i]
            
        return call_transcript

    except json.JSONDecodeError:
        print("Invalid JSON format. Please provide valid JSON data.")

In [139]:
# Example text data
text_data1 = '''
[
  {
    "Transcript": {
      "ParticipantId": "AGENT",
      "ParticipantRole": "AGENT",
      "Content": "Improving. Uh, Okay? Mm hmm. Obviously.",
      "BeginOffsetMillis": 1257,
      "EndOffsetMillis": 9697,
      "Sentiment": "NEUTRAL",
      "Id": "fb29489e-d06b-48b1-8e0b-519f17c4a68e"
    }
  },
  {
    "Transcript": {
      "ParticipantId": "CUSTOMER",
      "ParticipantRole": "CUSTOMER",
      "Content": "Fine. Thank you. I think",
      "BeginOffsetMillis": 3047,
      "EndOffsetMillis": 8055,
      "Sentiment": "POSITIVE",
      "Id": "7f6820ee-0487-4cbc-9b35-33e7c31e1d5d"
    }
  },
  {
    "Transcript": {
      "ParticipantId": "CUSTOMER",
      "ParticipantRole": "CUSTOMER",
      "Content": "A little bit. More even now.",
      "BeginOffsetMillis": 10457,
      "EndOffsetMillis": 16775,
      "Sentiment": "POSITIVE",
      "Id": "43d73132-0645-4133-8488-26aa9eac95bc"
    }
  }
]
'''

# Extract content from each transcript
content_list = extract_content_from_transcripts(text_data1)
print(content_list)

AGENT : Improving. Uh, Okay? Mm hmm. Obviously.
CUSTOMER : Fine. Thank you. I think
CUSTOMER : A little bit. More even now.


In [145]:
import json

def extract_content_from_transcripts(text_data):
    try:
        # Preprocess the text data to make it valid JSON
        text_data = text_data.replace("'", '"')
        text_data = text_data.replace("Transcript", '"Transcript"')
        text_data = text_data.replace("ParticipantId", '"ParticipantId"')
        text_data = text_data.replace("ParticipantRole", '"ParticipantRole"')
        text_data = text_data.replace("Content", '"Content"')
        text_data = text_data.replace("BeginOffsetMillis", '"BeginOffsetMillis"')
        text_data = text_data.replace("EndOffsetMillis", '"EndOffsetMillis"')
        text_data = text_data.replace("Sentiment", '"Sentiment"')
        text_data = text_data.replace("Id", '"Id"')

        # Convert the preprocessed text data to a list of dictionaries
        transcript_list = json.loads(text_data)

        # Initialize an empty list to store content from each transcript
        content_list = []

        # Iterate through each transcript dictionary and extract the content
        for transcript in transcript_list:
            content = transcript.get('Transcript', {}).get('Content')
            if content:
                content_list.append(content)

        return content_list

    except json.JSONDecodeError:
        print("Invalid JSON format. Please provide valid JSON data.")

# Extract content from each transcript
content_list = extract_content_from_transcripts(text_data)
print(content_list)

Invalid JSON format. Please provide valid JSON data.
None


In [147]:
import re

def extract_content_from_transcripts(text_data):
    try:
        # Define a regular expression pattern to extract content from each transcript
        pattern = r"'Content': '(.*?)'"

        # Use regular expression to find all matches of the pattern in the text data
        content_matches = re.findall(pattern, text_data)

        # Return the list of extracted content
        return content_matches

    except Exception as e:
        print("Error:", e)

# Example text data
text_data = '''
[
  {
    Transcript: {
      ParticipantId: 'AGENT',
      ParticipantRole: 'AGENT',
      Content: 'Improving. Uh, Okay? Mm hmm. Obviously.',
      BeginOffsetMillis: 1257,
      EndOffsetMillis: 9697,
      Sentiment: 'NEUTRAL',
      Id: 'fb29489e-d06b-48b1-8e0b-519f17c4a68e'
    }
  },
  {
    Transcript: {
      ParticipantId: 'CUSTOMER',
      ParticipantRole: 'CUSTOMER',
      Content: 'Fine. Thank you. I think',
      BeginOffsetMillis: 3047,
      EndOffsetMillis: 8055,
      Sentiment: 'POSITIVE',
      Id: '7f6820ee-0487-4cbc-9b35-33e7c31e1d5d'
    }
  },
  {
    Transcript: {
      ParticipantId: 'CUSTOMER',
      ParticipantRole: 'CUSTOMER',
      Content: 'A little bit. More even now.',
      BeginOffsetMillis: 10457,
      EndOffsetMillis: 16775,
      Sentiment: 'POSITIVE',
      Id: '43d73132-0645-4133-8488-26aa9eac95bc'
    }
  }
]
'''

# Extract content from each transcript
content_list = extract_content_from_transcripts(text_data)
print(content_list)


[]


In [144]:
print(preprocess_text_data(text_data))


[
  {"
    Transcript": {"
      ParticipantId": "AGENT",
      ParticipantRole": "AGENT",
      Content": "Improving. Uh, Okay? Mm hmm. Obviously.",
      BeginOffsetMillis": 1257,
      EndOffsetMillis": 9697,
      Sentiment": "NEUTRAL",
      Id": "fb29489e-d06b-48b1-8e0b-519f17c4a68e"
    }
  },
  {"
    Transcript": {"
      ParticipantId": "CUSTOMER",
      ParticipantRole": "CUSTOMER",
      Content": "Fine. Thank you. I think",
      BeginOffsetMillis": 3047,
      EndOffsetMillis": 8055,
      Sentiment": "POSITIVE",
      Id": "7f6820ee-0487-4cbc-9b35-33e7c31e1d5d"
    }
  },
  {"
    Transcript": {"
      ParticipantId": "CUSTOMER",
      ParticipantRole": "CUSTOMER",
      Content": "A little bit. More even now.",
      BeginOffsetMillis": 10457,
      EndOffsetMillis": 16775,
      Sentiment": "POSITIVE",
      Id": "43d73132-0645-4133-8488-26aa9eac95bc"
    }
  }
]



In [143]:
# Example text data
text_data = '''
[
  {
    Transcript: {
      ParticipantId: 'AGENT',
      ParticipantRole: 'AGENT',
      Content: 'Improving. Uh, Okay? Mm hmm. Obviously.',
      BeginOffsetMillis: 1257,
      EndOffsetMillis: 9697,
      Sentiment: 'NEUTRAL',
      Id: 'fb29489e-d06b-48b1-8e0b-519f17c4a68e'
    }
  },
  {
    Transcript: {
      ParticipantId: 'CUSTOMER',
      ParticipantRole: 'CUSTOMER',
      Content: 'Fine. Thank you. I think',
      BeginOffsetMillis: 3047,
      EndOffsetMillis: 8055,
      Sentiment: 'POSITIVE',
      Id: '7f6820ee-0487-4cbc-9b35-33e7c31e1d5d'
    }
  },
  {
    Transcript: {
      ParticipantId: 'CUSTOMER',
      ParticipantRole: 'CUSTOMER',
      Content: 'A little bit. More even now.',
      BeginOffsetMillis: 10457,
      EndOffsetMillis: 16775,
      Sentiment: 'POSITIVE',
      Id: '43d73132-0645-4133-8488-26aa9eac95bc'
    }
  }
]
'''

# Extract content from each transcript
content_list = extract_content_from_transcripts(text_data)
print(content_list)

Invalid JSON format. Please provide valid JSON data.
None


In [141]:
extracted_text_data = extract_content_from_transcripts(text_data)
print(extracted_text_data)

Invalid JSON format. Please provide valid JSON data.
None
